LogGpt

In [1]:
import os
from google.colab import userdata
from google.colab import drive

drive.mount("/content/drive")
# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
%cd "/content/drive/MyDrive/אונ' תל אביב"

Mounted at /content/drive
/content/drive/MyDrive/אונ' תל אביב


## Prepation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

## TinyLlama


1.   first attempt - train tiny Llama +   tinyllama-function-call-lora-adapter-250424
2.   failed - only output with nulls



In [3]:
!pip install lm-format-enforcer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
from peft     import PeftModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import \
        build_transformers_prefix_allowed_tokens_fn
from pydantic import BaseModel
from typing   import List, Dict, Optional
import torch, json, textwrap
from datasets import load_dataset

# ───────────────────────────────────────────────
# 1.  load TinyLlama-Chat + LoRA "function-call"
# ───────────────────────────────────────────────

max_seq_length = 4096 * 2
dtype = None

model, tok = FastLanguageModel.from_pretrained(
    model_name = "unclecode/tinyllama-function-call-lora-adapter-250424",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model)

In [ ]:


# ───────────────────────────────────────────────
# 2.  your JSON schema (Pydantic + LMFE)
# ───────────────────────────────────────────────
class CodeStat(BaseModel):
    code:int; count:int|str; bytes:int|str; description:str
class Summary(BaseModel):
    time_range: str
    status_codes: List[CodeStat]
    traffic_sources: Optional[Dict[str, str]] = None
    resource_types: Optional[Dict[str, str]] = None
    admin_activity: Optional[str] = None
    observations: Optional[Dict[str, str]] = None
    recommendations: Optional[List[str]] = None
class LogSynopsis(BaseModel):
    summary:Summary

parser       = JsonSchemaParser(LogSynopsis.model_json_schema())
inner_prefix = build_transformers_prefix_allowed_tokens_fn(tok, parser)

# ───────────────────────────────────────────────
# 3.  build prompt: function-call + log window
# ───────────────────────────────────────────────
def make_prompt(log_text:str)->str:

    # Best practice: Generate this directly from Pydantic to always stay in sync.
    # SCHEMA_JSON = LogSynopsis.model_json_schema()
    # For clarity, here is the corrected manual version:
    SCHEMA_JSON = {
      "type": "object",
      "properties": {
        "summary": {
          "type": "object",
          "properties": {
            "time_range": {"type": "string", "minLength": 5},
            "status_codes": {
              "type": "array", "minItems": 1,
              "items": {
                "type": "object",
                "properties": {
                  "code": {"type": "integer"},
                  "count": {"type": "integer"},
                  "bytes": {"type": "integer"},
                  "description": {"type": "string", "minLength": 1}
                },
                "required": ["code", "count", "bytes", "description"]
              }
            },
            # --- Optional fields below ---
            "traffic_sources": {"type": "object"},
            "resource_types": {"type": "object"},
            "admin_activity": {"type": "string"},
            "observations": {"type": "object"},
            "recommendations": {
              "type": "array",
              "items": {"type": "string"}
            }
          },
          # ✅ התיקון: רק השדות שהם באמת חובה לפי Pydantic
          "required": [
            "time_range",
            "status_codes"
          ]
        }
      },
      "required": ["summary"]
    }


    schema_block = textwrap.dedent(f"""\
        Call `summarize_log` with arguments matching:
        {json.dumps(SCHEMA_JSON, indent=2)}

    """)
    return f"{schema_block}\nLOG>>>\n{log_text.strip()}\n<<<"
# דוגמת לוג (קח מהקובץ שלך או קריאה מ-dataset)
from datasets import load_dataset
ds = load_dataset("json", data_files="log_alpaca3.jsonl", split="train")
log_window = ds[0]["input"]
out = ds[0]["output"]
prompt     = make_prompt(log_window)

# ───────────────────────────────────────────────
# 4.  tokenise prompt + prefix-fn להגבלת JSON
# ───────────────────────────────────────────────
batch       = tok([prompt], return_tensors="pt").to(model.device)
json_start  = batch["input_ids"].shape[-1]

def prefix_fn(batch_id, input_ids):
    if input_ids.shape[-1] < json_start:
        return list(range(tok.vocab_size))        # חופשי עד סוף-prompt
    return inner_prefix(batch_id, input_ids)      # אחרי-כן – רק מה שמותר בסכמה

# ───────────────────────────────────────────────
# 5.  generate
# ───────────────────────────────────────────────
with torch.no_grad():
    out_ids = model.generate(
        **batch,
        temperature = 0.2,
        max_new_tokens = 512,
        prefix_allowed_tokens_fn = prefix_fn,
        pad_token_id   = tok.eos_token_id,
    )

reply = tok.decode(out_ids[0][json_start:], skip_special_tokens=True)

# ───────────────────────────────────────────────
# 6.  validate JSON
# ───────────────────────────────────────────────
data = json.loads(reply)                     # json.loads יזרוק אם לא תקין
LogSynopsis.model_validate(data)             # Pydantic validation
print("\n✅ Parsed & validated:", json.dumps(data, indent=2, ensure_ascii=False))
print("---"*100)
print(out)


✅ Parsed & validated: {
  "summary": {
    "status_codes": [],
    "observations": null,
    "traffic_sources": null,
    "time_range": ""
  }
}
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{"summary":{"time_range":"28/Jul/2025","status_codes":[{"code":200,"count":19,"description":"Successful requests"},{"code":301,"count":7,"description":"Moved Permanently"},{"code":302,"count":6,"description":"Found (Temporary Redirect)"}],"traffic_sources":null,"resource_types":null,"admin_activity":null,"observations":null,"recommendations":["Monitor the frequency of 301 redirects to ensure they are necessary.","Review the user agent strings for accuracy and relevance."]}}


## Train TinyLlama from scartch


In [ ]:
import torch
import json
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from datasets import load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

# Define the maximum sequence length for the model
max_seq_length = 8192

# ───────────────────────────────────────────────
# 1. Define the Pydantic Schema and Tool Structure
# ───────────────────────────────────────────────
class CodeStat(BaseModel):
    code:int
    count:int
    bytes:int
    description:str

class Summary(BaseModel):
    time_range: str
    status_codes: List[CodeStat]
    traffic_sources: Optional[Dict[str, str]] = None
    resource_types: Optional[Dict[str, str]] = None
    admin_activity: Optional[str] = None
    observations: Optional[Dict[str, str]] = None
    recommendations: Optional[List[str]] = None

class LogSynopsis(BaseModel):
    summary:Summary

# Define the full tool structure with a name.
LOG_SUMMARY_TOOL = {
    "name": "log_summary_tool",
    "description": "Extracts a structured summary from a block of web server log text.",
    "parameters": LogSynopsis.model_json_schema()
}
TOOLS_LIST_STR = json.dumps([LOG_SUMMARY_TOOL], indent=4)


# ───────────────────────────────────────────────
# 2. Load the BASE model and add a NEW adapter
# ───────────────────────────────────────────────
# We start from the original base chat model for a clean fine-tune.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit", # Use the specified 4-bit model
    max_seq_length = max_seq_length,
)
tokenizer.pad_token = tokenizer.eos_token

# We add a fresh LoRA adapter for our specific task.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)


# ───────────────────────────────────────────────
# 3. Prepare the Dataset with the Correct Prompt Format
# ───────────────────────────────────────────────
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    system_instruction = "You are a helpful assistant with access to the following functions. Use them if required -"

    for log_text, output_str in zip(inputs, outputs):
        try:
            parsed_arguments = json.loads(output_str)
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON in output: {output_str}")
            continue

        full_tool_call_output = {
            "name": "log_summary_tool",
            "arguments": parsed_arguments
        }
        clean_output_json = json.dumps(full_tool_call_output)

        # This is the standard chat template for TinyLLaMA
        system_prompt = f"<|system|>\n{system_instruction}\n{TOOLS_LIST_STR}</s>"
        user_prompt = f"<|user|>\n{log_text}</s>"
        assistant_response = f"<|assistant|>\n<functioncall> {clean_output_json}</s>"

        full_text = system_prompt + "\n" + user_prompt + "\n" + assistant_response
        texts.append(full_text)

    return { "text" : texts, }

# ✅ CORRECTED: Load both JSONL files into a single dataset
data_files = ["log_alpaca3.jsonl", "log_alpaca.jsonl"]
dataset = load_dataset("json", data_files=data_files, split="train")
dataset = dataset.map(formatting_prompts_func, batched=True,)


# ───────────────────────────────────────────────
# 4. Define Training Arguments
# ───────────────────────────────────────────────
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,       # ✅ Use epochs instead of max_steps for stability
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

# ───────────────────────────────────────────────
# 5. Start the Fine-Tuning Process
# ───────────────────────────────────────────────
print("Starting fine-tuning from base model...")
trainer.train()
print("Fine-tuning complete!")

# ───────────────────────────────────────────────
# 6. Save the new adapter
# ───────────────────────────────────────────────
print("Saving the new LoRA adapter to 'my-log-finetune-lora'...")
model.save_pretrained("my-log-finetune-lora")
print("Adapter saved! You can now use the updated inference script.")


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/600 [00:00<?, ? examples/s]

Starting fine-tuning from base model...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 5 | Total steps = 375
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 12,615,680 of 1,112,664,064 (1.13% trained)


Step,Training Loss
1,0.667900
2,0.667900
3,0.668900
4,0.668200
5,0.667300
6,0.665200
7,0.663100
8,0.659600
9,0.656500
10,0.652700


Fine-tuning complete!
Saving the new LoRA adapter to 'my-log-finetune-lora'...
Adapter saved! You can now use the updated inference script.


In [ ]:
import torch
import json
import textwrap
from pydantic import BaseModel, Field
from typing import List, Dict, Optional, Any
from datasets import load_dataset
from unsloth import FastLanguageModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn

# ───────────────────────────────────────────────
# 1. Load your FINE-TUNED model
# ───────────────────────────────────────────────
# This loads the adapter that you just trained and saved.
max_seq_length = 8192
dtype = None # None for auto-detection
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "my-log-finetune-lora", # ✅ Load your custom adapter!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# ───────────────────────────────────────────────
# 2. Define Pydantic Schemas & Tool Structure
# ───────────────────────────────────────────────
# These schemas define the structure of the *arguments* for our tool.
class CodeStat(BaseModel):
    code:int
    count:int
    bytes:int
    description:str

class Summary(BaseModel):
    time_range: str
    status_codes: List[CodeStat]
    traffic_sources: Optional[Dict[str, str]] = None
    resource_types: Optional[Dict[str, str]] = None
    admin_activity: Optional[str] = None
    observations: Optional[Dict[str, str]] = None
    recommendations: Optional[List[str]] = None

class LogSynopsis(BaseModel):
    summary:Summary

# Define the full tool structure that the model expects.
LOG_SUMMARY_TOOL = {
    "name": "log_summary_tool",
    "description": "Extracts a structured summary from a block of web server log text.",
    "parameters": LogSynopsis.model_json_schema()
}
TOOLS_LIST_STR = json.dumps([LOG_SUMMARY_TOOL], indent=4)

# Create a Pydantic model for the *entire tool call* output.
# This is what the lm-format-enforcer will validate against.
class ToolCall(BaseModel):
    name: str = Field(..., description="The name of the function to call.")
    arguments: LogSynopsis = Field(..., description="The arguments to pass to the function.")


# ───────────────────────────────────────────────
# 3. Setup the Format Enforcer
# ───────────────────────────────────────────────
# The parser now uses the schema of the full ToolCall.
parser = JsonSchemaParser(ToolCall.model_json_schema())
prefix_enforcer = build_transformers_prefix_allowed_tokens_fn(tokenizer, parser)

# ───────────────────────────────────────────────
# 4. Build the Inference Prompt
# ───────────────────────────────────────────────
# This prompt now perfectly matches the standard TinyLLaMA format
# used in the corrected training script.
def make_inference_prompt(log_text:str)->str:
    system_instruction = "You are a helpful assistant with access to the following functions. Use them if required -"

    system_prompt = f"<|system|>\n{system_instruction}\n{TOOLS_LIST_STR}</s>"
    user_prompt = f"<|user|>\n{log_text}</s>"
    assistant_prompt = f"<|assistant|>\n<functioncall> " # We prompt it to start the function call

    return system_prompt + "\n" + user_prompt + "\n" + assistant_prompt


# ───────────────────────────────────────────────
# 5. Run Inference
# ───────────────────────────────────────────────
# Load the same dataset to get an example log to test with
ds = load_dataset("json", data_files="log_alpaca3.jsonl", split="train")
log_window = ds[0]["input"]
ground_truth_output = json.loads(ds[0]["output"])

# Create the prompt and tokenize it
prompt = make_inference_prompt(log_window)
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
prompt_len = inputs["input_ids"].shape[1]

# Generate the output, using the format enforcer as a safety rail
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens = 1024,
        prefix_allowed_tokens_fn = prefix_enforcer,
        pad_token_id = tokenizer.eos_token_id,
    )

# Decode only the newly generated tokens
newly_generated_tokens = outputs[0][prompt_len:]
json_part = tokenizer.decode(newly_generated_tokens, skip_special_tokens=True)


# ───────────────────────────────────────────────
# 6. Validate and Compare the Output
# ───────────────────────────────────────────────
print("✅ Model Generated JSON:")
try:
    json_part = json_part.replace(tokenizer.eos_token, "").strip()
    # First, validate the entire tool call structure
    tool_call_data = ToolCall.model_validate_json(json_part)

    # If successful, print the nicely formatted arguments
    # ✅ CORRECTED: Switched from deprecated .dict() to .model_dump()
    print(json.dumps(tool_call_data.arguments.model_dump(), indent=2, ensure_ascii=False))

except (json.JSONDecodeError, ValueError) as e:
    print(f"Error parsing or validating generated JSON: {e}")
    print(f"Raw model output: {json_part}")


print("\n" + "---"*20 + "\n")

print("🎯 Ground Truth JSON:")
print(json.dumps(ground_truth_output, indent=2, ensure_ascii=False))


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model Generated JSON:
{
  "summary": {
    "time_range": "- ",
    "status_codes": [],
    "traffic_sources": {
      "TSTSTSTSTS  Jones - - - - Jones ": ") "
    },
    "resource_types": {
      " }, ": " ",
      " ": ", . "
    },
    "admin_activity": " ",
    "observations": {
      " ": " "
    },
    "recommendations": [
      " ",
      " ",
      " Jones Jones Jones Jones Jones ",
      " ",
      " ",
      " ",
      " ",
      " ",
      " ",
      " ",
      " ",
      " ",
      " ",
      " Jones ",
      " Jones ",
     

## Train strognger Model - llama-3-8b-bnb-4bit





In [ ]:
import torch
import json
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from datasets import load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

# Define the maximum sequence length for the model
max_seq_length = 8192

# ───────────────────────────────────────────────
# 1. Define the Pydantic Schema and Tool Structure
# ───────────────────────────────────────────────
class CodeStat(BaseModel):
    code:int
    count:int
    bytes:int
    description:str

class Summary(BaseModel):
    time_range: str
    status_codes: List[CodeStat]
    traffic_sources: Optional[Dict[str, str]] = None
    resource_types: Optional[Dict[str, str]] = None
    admin_activity: Optional[str] = None
    observations: Optional[Dict[str, str]] = None
    recommendations: Optional[List[str]] = None

class LogSynopsis(BaseModel):
    summary:Summary

# Define the full tool structure with a name.
LOG_SUMMARY_TOOL = {
    "name": "log_summary_tool",
    "description": "Extracts a structured summary from a block of web server log text.",
    "parameters": LogSynopsis.model_json_schema()
}
TOOLS_LIST_STR = json.dumps([LOG_SUMMARY_TOOL], indent=4)


# ───────────────────────────────────────────────
# 2. Load the Llama 3 8B BASE model and add a NEW adapter
# ───────────────────────────────────────────────
# ✅ CORRECTED: Switched to a more powerful base model.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
)

# We add a fresh LoRA adapter for our specific task.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)


# ───────────────────────────────────────────────
# 3. Prepare the Dataset with the Correct Llama 3 Prompt Format
# ───────────────────────────────────────────────
# ✅ CORRECTED: Switched back to the Llama 3 chat template.
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    system_instruction = "You are a helpful assistant with access to the following functions. Use them if required -"

    for log_text, output_str in zip(inputs, outputs):
        try:
            parsed_arguments = json.loads(output_str)
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON in output: {output_str}")
            continue

        full_tool_call_output = {
            "name": "log_summary_tool",
            "arguments": parsed_arguments
        }
        clean_output_json = json.dumps(full_tool_call_output)

        # This is the standard chat template for Llama 3
        prompt = f"<|start_header_id|>system<|end_header_id|>\n\n"
        prompt += f"{system_instruction}\n{TOOLS_LIST_STR}<|eot_id|>"
        prompt += f"<|start_header_id|>user<|end_header_id|>\n\n"
        prompt += f"{log_text}<|eot_id|>"
        prompt += f"<|start_header_id|>assistant<|end_header_id|>\n\n"
        full_text = f"{prompt}<functioncall>{clean_output_json}<|eot_id|>"
        texts.append(full_text)

    return { "text" : texts, }

# Load both JSONL files into a single dataset
data_files = ["log_alpaca3.jsonl", "log_alpaca.jsonl"]
dataset = load_dataset("json", data_files=data_files, split="train")
dataset = dataset.map(formatting_prompts_func, batched=True,)

# ───────────────────────────────────────────────
# 4. Define Training Arguments
# ───────────────────────────────────────────────
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, # 3 epochs is a good starting point for a larger model
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

# ───────────────────────────────────────────────
# 5. Start the Fine-Tuning Process
# ───────────────────────────────────────────────
print("Starting fine-tuning from Llama 3 8B base model...")
trainer.train()
print("Fine-tuning complete!")

# ───────────────────────────────────────────────
# 6. Save the new adapter
# ───────────────────────────────────────────────
print("Saving the new LoRA adapter to 'my-log-finetune-lora'...")
model.save_pretrained("my-log-finetune-lora")
print("Adapter saved! You can now use the updated inference script.")


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/600 [00:00<?, ? examples/s]

Starting fine-tuning from Llama 3 8B base model...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 3 | Total steps = 225
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,1.197300
2,0.969500
3,1.035200
4,0.909300
5,1.031400
6,1.078500
7,1.008400
8,1.072100
9,0.953100
10,0.985600


Fine-tuning complete!
Saving the new LoRA adapter to 'my-log-finetune-lora'...
Adapter saved! You can now use the updated inference script.


In [ ]:
import torch
import json
import textwrap
from pydantic import BaseModel, Field
from typing import List, Dict, Optional, Any
from datasets import load_dataset
from unsloth import FastLanguageModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn

# ───────────────────────────────────────────────
# 1. Load your FINE-TUNED model
# ───────────────────────────────────────────────
# This loads the adapter that you just trained and saved.
max_seq_length = 8192
dtype = None # None for auto-detection
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "my-log-finetune-lora", # ✅ Load your custom Llama 3 adapter!
#     max_seq_length = max_seq_length,
#     rope_scaling= 4,
#     dtype = dtype,
#     load_in_4bit = True,
# )
# FastLanguageModel.for_inference(model)

# ───────────────────────────────────────────────
# 2. Define Pydantic Schemas & Tool Structure
# ───────────────────────────────────────────────
# These schemas define the structure of the *arguments* for our tool.
class CodeStat(BaseModel):
    code:int
    count:int
    bytes:int
    description:str

class Summary(BaseModel):
    time_range: str
    status_codes: List[CodeStat]
    traffic_sources: Optional[Dict[str, str]] = None
    resource_types: Optional[Dict[str, str]] = None
    admin_activity: Optional[str] = None
    observations: Optional[Dict[str, str]] = None
    recommendations: Optional[List[str]] = None

class LogSynopsis(BaseModel):
    summary:Summary

# Define the full tool structure that the model expects.
LOG_SUMMARY_TOOL = {
    "name": "log_summary_tool",
    "description": "Extracts a structured summary from a block of web server log text.",
    "parameters": LogSynopsis.model_json_schema()
}
TOOLS_LIST_STR = json.dumps([LOG_SUMMARY_TOOL], indent=4)

# Create a Pydantic model for the *entire tool call* output.
# This is what the lm-format-enforcer will validate against.
class ToolCall(BaseModel):
    name: str = Field(..., description="The name of the function to call.")
    arguments: LogSynopsis = Field(..., description="The arguments to pass to the function.")


# ───────────────────────────────────────────────
# 3. Setup the Format Enforcer
# ───────────────────────────────────────────────
# The parser now uses the schema of the full ToolCall.
parser = JsonSchemaParser(ToolCall.model_json_schema())
prefix_enforcer = build_transformers_prefix_allowed_tokens_fn(tokenizer, parser)

# ───────────────────────────────────────────────
# 4. Build the Inference Prompt
# ───────────────────────────────────────────────
# This prompt now perfectly matches the Llama 3 format used in training.
def make_inference_prompt(log_text:str)->str:
    system_instruction = "You are a helpful assistant with access to the following functions. Use them if required -"

    prompt = f"<|start_header_id|>system<|end_header_id|>\n\n"
    prompt += f"{system_instruction}\n{TOOLS_LIST_STR}<|eot_id|>"
    prompt += f"<|start_header_id|>user<|end_header_id|>\n\n"
    prompt += f"{log_text}<|eot_id|>"
    prompt += f"<|start_header_id|>assistant<|end_header_id|>\n\n"
    prompt += f"<functioncall>" # We prompt it to start the function call
    return prompt


# ───────────────────────────────────────────────
# 5. Run Inference
# ───────────────────────────────────────────────
# Load the same dataset to get an example log to test with
ds = load_dataset("json", data_files="log_alpaca3.jsonl", split="train")
log_window = ds[2]["input"]
ground_truth_output = json.loads(ds[0]["output"])

# Create the prompt and tokenize it
prompt = make_inference_prompt(log_window)
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
prompt_len = inputs["input_ids"].shape[1]

# Generate the output, using the format enforcer as a safety rail
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        temperature = 0.1,
        max_new_tokens = 1024,
        prefix_allowed_tokens_fn = prefix_enforcer,
        pad_token_id = tokenizer.eos_token_id,
    )

# Decode only the newly generated tokens
newly_generated_tokens = outputs[0][prompt_len:]
json_part = tokenizer.decode(newly_generated_tokens, skip_special_tokens=True)


# ───────────────────────────────────────────────
# 6. Validate and Compare the Output
# ───────────────────────────────────────────────
print("✅ Model Generated JSON:")
try:
    # The output from the model should be the JSON content of the function call
    json_part = json_part.replace("<|eot_id|>", "").strip()

    # First, validate the entire tool call structure
    tool_call_data = ToolCall.model_validate_json(json_part)

    # If successful, print the nicely formatted arguments
    print(json.dumps(tool_call_data.arguments.model_dump(), indent=2, ensure_ascii=False))

except (json.JSONDecodeError, ValueError) as e:
    print(f"Error parsing or validating generated JSON: {e}")
    print(f"Raw model output: {json_part}")


print("\n" + "---"*20 + "\n")

print("🎯 Ground Truth JSON:")
print(json.dumps(ground_truth_output, indent=2, ensure_ascii=False))


✅ Model Generated JSON:
Error parsing or validating generated JSON: 1 validation error for ToolCall
  Invalid JSON: control character (\u0000-\u001F) found while parsing a string at line 30 column 1193 [type=json_invalid, input_value='{"arguments" : {"summary...DDDDD*DD**, <DDDD: <* >', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
Raw model output: {"arguments" : {"summary" :   {"status_codes"  : 	   

 	   [] 	   	   	,"admin_activity"








			:" " ,"observations" 	   	   


: null 	   
 	   
,"resource_types" 
: null ,"time_range"								



:" "


									,"recommendations"
											:			    [ ]	    							}							   		}   									,"name"



   	   	:" Jones ( Jones Jones Jones Jones Jones < Jones < < < < < <TS < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < < <<< < < < < < < < < < <<<< < < < < < < <TP < < < <TPs <TP <TP <TP <

## New Try - diffrent data sturcture format

In [8]:
import torch
import json
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from datasets import load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

# Define the maximum sequence length for the model
max_seq_length = 8192

# ───────────────────────────────────────────────
# 1. Define the Pydantic Schema (No changes needed)
# ───────────────────────────────────────────────
class CodeStat(BaseModel):
    code:int
    count:int
    bytes:int
    description:str

class Summary(BaseModel):
    time_range: str
    status_codes: List[CodeStat]
    traffic_sources: Optional[Dict[str, str]] = None
    resource_types: Optional[Dict[str, str]] = None
    admin_activity: Optional[str] = None
    observations: Optional[Dict[str, str]] = None
    recommendations: Optional[List[str]] = None

class LogSynopsis(BaseModel):
    summary:Summary

# ───────────────────────────────────────────────
# 2. Load the Llama 3 8B BASE model and add a NEW adapter
# ───────────────────────────────────────────────
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)


# ───────────────────────────────────────────────
# 3. Prepare the Dataset with the SIMPLIFIED Alpaca Prompt Format
# ───────────────────────────────────────────────
# ✅ CORRECTED: Switched to a simple, direct instruction format.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# We still need the schema to pass into the instruction.
LOG_SYNOPSIS_SCHEMA = LogSynopsis.model_json_schema()

def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    instruction = (
        "You are NetLogGPT. Summarize the following log window and respond ONLY with a valid JSON object that conforms to this Pydantic schema:\n"
        f"{json.dumps(LOG_SYNOPSIS_SCHEMA, indent=2)}"
    )

    for log_text, output_str in zip(inputs, outputs):
        try:
            # The output from the dataset needs to be a clean JSON string
            parsed_output = json.loads(output_str)
            clean_output_json = json.dumps(parsed_output)
        except json.JSONDecodeError:
            continue

        full_text = alpaca_prompt.format(
            instruction,
            log_text,
            clean_output_json + tokenizer.eos_token # Add EOS token to signal end of generation
        )
        texts.append(full_text)

    return { "text" : texts, }

# Load both JSONL files into a single dataset
data_files = ["log_alpaca3.jsonl", "log_alpaca.jsonl"]
dataset = load_dataset("json", data_files=data_files, split="train")
dataset = dataset.map(formatting_prompts_func, batched=True,)

# ───────────────────────────────────────────────
# 4. Define Training Arguments
# ───────────────────────────────────────────────
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10, # Increased warmup for a larger dataset/model
        num_train_epochs = 4,
        learning_rate = 1e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",


    ),
)

# ───────────────────────────────────────────────
# 5. Start the Fine-Tuning Process
# ───────────────────────────────────────────────
print("Starting fine-tuning with simplified Alpaca format...")
trainer.train(resume_from_checkpoint=True)
print("Fine-tuning complete!")

# ───────────────────────────────────────────────
# 6. Save the new adapter
# ───────────────────────────────────────────────
print("Saving the new LoRA adapter to 'my-log-finetune-lora'...")
model.save_pretrained("my-log-finetune-lora-new")
print("Adapter saved!")


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/600 [00:00<?, ? examples/s]

Starting fine-tuning with simplified Alpaca format...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 4 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
226,0.575000
227,0.587400
228,0.638100
229,0.552600
230,0.572300
231,0.585200
232,0.633300
233,0.531800
234,0.543800
235,0.510700


Fine-tuning complete!
Saving the new LoRA adapter to 'my-log-finetune-lora'...
Adapter saved!


In [9]:
import torch
import json
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from datasets import load_dataset
from unsloth import FastLanguageModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn

# ───────────────────────────────────────────────
# 1. Load your FINE-TUNED model
# ───────────────────────────────────────────────
# This loads the adapter that you just trained with the Alpaca format.
max_seq_length = 8192
dtype = None # None for auto-detection
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "my-log-finetune-lora-new", # ✅ Load your custom Llama 3 adapter!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# ───────────────────────────────────────────────
# 2. Define Pydantic Schemas
# ───────────────────────────────────────────────
class CodeStat(BaseModel):
    code:int
    count:int
    bytes:int
    description:str

class Summary(BaseModel):
    time_range: str
    status_codes: List[CodeStat]
    traffic_sources: Optional[Dict[str, str]] = None
    resource_types: Optional[Dict[str, str]] = None
    admin_activity: Optional[str] = None
    observations: Optional[Dict[str, str]] = None
    recommendations: Optional[List[str]] = None

class LogSynopsis(BaseModel):
    summary:Summary

# ───────────────────────────────────────────────
# 3. Setup the Format Enforcer
# ───────────────────────────────────────────────
# The parser now directly enforces the LogSynopsis schema, as we are no longer using a tool/function call structure.
parser = JsonSchemaParser(LogSynopsis.model_json_schema())
prefix_enforcer = build_transformers_prefix_allowed_tokens_fn(tokenizer, parser)

# ───────────────────────────────────────────────
# 4. Build the Inference Prompt
# ───────────────────────────────────────────────
# This prompt now perfectly matches the Alpaca format used in the final training script.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

LOG_SYNOPSIS_SCHEMA = LogSynopsis.model_json_schema()

def make_inference_prompt(log_text:str)->str:
    instruction = (
        "You are NetLogGPT. Summarize the following log window and respond ONLY with a valid JSON object that conforms to this Pydantic schema:\n"
        f"{json.dumps(LOG_SYNOPSIS_SCHEMA, indent=2)}"
    )
    return alpaca_prompt.format(
        instruction,
        log_text,
        "", # The response to be generated
    )


# ───────────────────────────────────────────────
# 5. Run Inference
# ───────────────────────────────────────────────
# Load the same dataset to get an example log to test with
ds = load_dataset("json", data_files="log_alpaca3.jsonl", split="train")
log_window = ds[2]["input"]
ground_truth_output = json.loads(ds[2]["output"])

# Create the prompt and tokenize it
prompt = make_inference_prompt(log_window)
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
prompt_len = inputs["input_ids"].shape[1]

# Generate the output, using the format enforcer as a safety rail
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens = 512,
        prefix_allowed_tokens_fn = prefix_enforcer,
        pad_token_id = tokenizer.eos_token_id,
    )

# Decode only the newly generated tokens
newly_generated_tokens = outputs[0][prompt_len:]
json_part = tokenizer.decode(newly_generated_tokens, skip_special_tokens=True)


# ───────────────────────────────────────────────
# 6. Validate and Compare the Output
# ───────────────────────────────────────────────
print("✅ Model Generated JSON:")
try:
    # The output from the model should be the JSON content of the function call
    json_part = json_part.replace("<|eot_id|>", "").strip()

    # Validate the JSON against our Pydantic schema
    validated_data = LogSynopsis.model_validate_json(json_part)

    # If successful, print the nicely formatted arguments
    print(validated_data.model_dump_json(indent=2))

except (json.JSONDecodeError, ValueError) as e:
    print(f"Error parsing or validating generated JSON: {e}")
    print(f"Raw model output: {json_part}")


print("\n" + "---"*20 + "\n")

print("🎯 Ground Truth JSON:")
print(json.dumps(ground_truth_output, indent=2, ensure_ascii=False))


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model Generated JSON:
Error parsing or validating generated JSON: 1 validation error for LogSynopsis
  Invalid JSON: EOF while parsing a value at line 90 column 49 [type=json_invalid, input_value='{\n    "summary": {\n   ..."Web Server Responses",', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
Raw model output: {
    "summary": {
        "time_range": "2025/07/28 11:51-11:52 IST",
        "status_codes": [
           {
           "description": "Successful Responses",
           "code